# 1. Build a convolutional neural network to classify object proposals (N+1 classes)

In [1]:
import json
from sklearn.model_selection import train_test_split
from Dataset import PotholeDataset
from tqdm import tqdm
from torchvision import transforms

# Load splits.json
with open('../Potholes/splits.json', 'r') as f:
    splits = json.load(f)

# Extract train and test splits
train_files = splits['train']
test_files = splits['test']

# Further split the train set into train and validation
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

In [2]:

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# Paths
img_dir = '../Potholes/annotated-images'
annotations_dir = '../Potholes/labeled_proposals'

# Initialize datasets
train_dataset = PotholeDataset(img_dir, annotations_dir, train_files, transform=transform)
val_dataset = PotholeDataset(img_dir, annotations_dir, val_files, transform=transform)
test_dataset = PotholeDataset(img_dir, annotations_dir, test_files, transform=transform)

Loading annotations: 100%|██████████| 133/133 [00:00<00:00, 2158.47it/s]


In [4]:
from OurDataloader import balanced_loader
train_loader = balanced_loader(train_dataset, batch_size=32, weight_sample = True)
val_loader = balanced_loader(val_dataset, batch_size=32, shuffle=False, weight_sample = True)
test_loader = balanced_loader(test_dataset, batch_size=32, shuffle=False)

len_train_data = len(train_loader.dataset.data)
len_train_label = len(train_loader.dataset.labels)
print(len_train_data)
print(len_train_label)

142273
142273


In [ ]:
import torch
from PotholeCNN import PotholeCNN
import torch.nn.functional as F
model = PotholeCNN()
# Assume `train_loader` is already defined and contains batches of data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Get a single batch from the DataLoader
batch = next(iter(train_loader))
inputs, labels = batch[0].to(device), batch[1].to(device)

# Forward pass
outputs = model(inputs).squeeze()  # Remove singleton dimension if needed

# Compute the loss
loss = F.binary_cross_entropy_with_logits(outputs, labels)

# Print the computed loss
print(f"Test Loss on a single batch: {loss.item()}")



 67%|██████▋   | 2970/4447 [05:12<02:35,  9.52it/s]


KeyboardInterrupt: 

In [6]:
import torch
from PotholeCNN import PotholeCNN
from TrainLoop import train
# Initialize model, optimizer, and load data
model = PotholeCNN()  # Your model class
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train(model, optimizer, epochs=10, train_loader=train_loader, test_loader=test_loader, device=device)

100%|█████████▉| 4446/4447 [09:25<00:00,  7.86it/s]


ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([]))